In [1]:
import pandas as pd
import numpy as np
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from source.acquire_m import *
from source.model_marley import *
from scipy import stats as scs


%load_ext autoreload
%autoreload 2

RDKit WARNING: [09:13:33] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
# This cell takes in a disease we wish to target
wrangle_obj = wrangle('Coronavirus')

In [3]:
#This cell searches th ChEMBL database for the disease,
# it will return a df of single protien targets. Single protien targets are 
# important because these are what were are going to compare the chemical compounds to.

# *NOTE* This is the first acquire stage of aqcuire. Because of the complexity of this data 
# we must acquire twice, The first acquistion will be to find a target disease, or virus.
# The second aqcuisition will be using the chosen single protien target to find the chemical compounds tested
# against it, this is the ultimate end goal of the aqcuire and prepare steps.

# *NOTE* Because each protien is different we are providing a way to test each protien from the df
# in the df. They will each achieve a different result. 

acquire_df = wrangle_obj.acquire_data()
acquire_df

,index,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,0,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.000,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
1,1,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.000,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
2,2,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.000,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


In [4]:
# This cell provides a list of ChEMBL ID's. The user (that's you) will copy one of the id's
# and paste it into the cell below, This function will create the dataframe for this specific 
# single protien target and is whht will be used the rest of the notebook. If the user wants to run
# a different single protien target rerun this cell and paste a seperate ChEMBL id in the
# input

bioactivity_df = wrangle_obj.get_bioactivity_data()
bioactivity_df

List of single protein ChEMBL ID's from chosen disease:
0       CHEMBL3927
1       CHEMBL5118
2    CHEMBL4523582
Name: target_chembl_id, dtype: object
Input a single protein target ID from the list:CHEMBL3927


,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
activity_comment,,,,,,,,,,,,,,,,,,,,,
NaN,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,7.200
NaN,1480936,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,9.400
NaN,1481061,[],CHEMBL830868,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,13.500
NaN,1481065,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,13.110
NaN,1481066,[],CHEMBL829584,In vitro inhibitory concentration against SARS...,B,NaN,NaN,BAO_0000190,BAO_0000357,single protein format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,12041507,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,NaN,NaN,BAO_0000190,BAO_0000019,assay format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,10.600
NaN,12041508,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,NaN,NaN,BAO_0000190,BAO_0000019,assay format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,10.100
NaN,12041509,[],CHEMBL2150313,Inhibition of SARS-CoV PLpro expressed in Esch...,B,NaN,NaN,BAO_0000190,BAO_0000019,assay format,...,SARS coronavirus,SARS coronavirus 3C-like proteinase,227859,NaN,NaN,IC50,uM,UO_0000065,NaN,11.500


In [5]:
#This cell is the second stage of the acquire and aquistion process. Here we will create the dataframes and csv's 
# used  to look at the chemical coumpounds tested against.

# Below you will find a kruskal-wallis H test run  
wrangle_obj.prepare_dataframe()

TypeError: '<' not supported between instances of 'float' and 'str'